# 合并other和street

In [1]:
from data.inputs_mono_other import orders as o1
from data.inputs_mono_street import orders as o2
from data.inputs_poly_other import orders as o3
from data.inputs_poly_street import orders as o4
def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
def merge_list(l1, l2, path):
    both = list(set(l1+l2))
    save_list(both, path)
merge_list(o1, o2, 'data/inputs_mono_unique.py')
merge_list(o3, o4, 'data/inputs_poly_unique.py')

# 合并单双色

In [1]:
from data.inputs_mono_other import orders as mono
from data.inputs_poly_other import orders as poly
# 合并单双色
def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
def unique_list(l):
    return list(set(l))

# merge_list(list(mono.values())[:18], list(poly.values())[:2], 'data/inputs_unique_exp.py')

# mono_list = unique_list(list(mono.values()))
# poly_list = unique_list(list(poly.values()))
mono_list = unique_list(mono)
poly_list = unique_list(poly)

n = len(mono_list+poly_list)
# 以10个为单位，每10个指令包含8个单色指令，2个双色指令
x=8
y=2
m=0
p=0
orders = []
for i in range(0, n, 10):
    try:
        orders[i:i+10] = mono_list[m:m+x]+poly_list[p:p+y]
        m+=x
        p+=y
    except:
        orders[i:i+10] = mono_list[m:]+poly_list[p:]
save_list(orders, 'data/inputs_unique.py')

### 按照一定比例进行指令合并

In [1]:
from data.structured_input_1 import orders as raw_orders1
from data.structured_input_2 import orders as raw_orders2
import random

orders1 = list(raw_orders1.values())
orders2 = list(raw_orders2.values())
def unique_list(l):
    return list(set(l))

# 以10个为单位，每10个指令包含8个单色指令，2个双色指令
def compose(a_list, b_list):
    n = len(a_list+b_list)
    x=8
    y=2
    m=0
    p=0
    orders = []
    for i in range(0, n, 10):
        try:
            orders[i:i+10] = a_list[m:m+x]+b_list[p:p+y]
            m+=x
            p+=y
        except:
            orders[i:i+10] = a_list[m:]+b_list[p:]
    return orders

orders = compose(orders1, orders2)

def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
save_list(orders, 'data/structure_input_all.py')

# 数据增强

## 反色

In [1]:
from inputs_unique import orders
orders = list(orders.values())
reversed = []
def color_reverse(order):
    if "red" in order or "blue" in order:
        if "red" in order and "blue" in order:
            temp_text = order.replace("red", "TEMP_REDA")
            temp_text = temp_text.replace("blue", "red")
            order = temp_text.replace("TEMP_REDA", "blue")
        elif "red" in order:
            order = order.replace("red", "blue")
        elif "blue" in order:
            order = order.replace("blue", "red")
    elif "Red" in order or "Blue" in order:
        if "Red" in order and "Blue" in order:
            temp_text = order.replace("Red", "TEMP_REDA")
            temp_text = temp_text.replace("Blue", "Red")
            order = temp_text.replace("TEMP_REDA", "Blue")
        elif "Red" in order:
            order = order.replace("Red", "Blue")
        elif "Blue" in order:
            order = order.replace("Blue", "Red")
    return order
for order in orders:
    reversed.append(color_reverse(order))

def merge_list(l1, l2, path):
    both = list(set(l1+l2))
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(both):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
merge_list(orders, reversed, 'inputs_unique.py')


: 

## 去除周围实体过多的部署指令

In [1]:

# bad_loc0 = [['Station 2x heavy machine gun nests of the Red Army at the corners of the intersection of West 33rd Street and Broadway,while deploying 7x anti-aircraft guns of the Blue Army in a circular formation around the 34th Street–Herald Square station.'],
#  ['Deploy 6x rocket launchers of the Red Army in a hexagonal formation around the center of Times Square.'],
#  ['Deploy 2x heavy machine gun nests of the Blue Army at the corners of the intersection of West 33rd Street and Broadway.'],
#  ['Position 8x mobile air defense systems of the Red Army in a circular formation around the center of Times Square.']]
# bad_loc1 = [['Station 5x mobile missile launchers of the Blue Army in a triangular formation near the intersection of Broadway and West 34th Street.'],
#  ['Position 3x infantry fighting vehicles of the Red Army in a linear formation along Broadway,facing north,while deploying 5x snipers of the Blue Army on the rooftops of buildings along West 32nd Street.'],
#  ['Deploy 5x mobile command centers of the Red Army in a triangular formation near the intersection of Broadway and West 32nd Street.']]
bad_loc = ['红军在环南路309巷9弄部署三辆CM-21装甲运兵车，呈V字形队形。',
 '蓝军在远传电信店铺西侧20米部署3架RQ-4全球鹰高空侦察无人机，呈一字形编队，间距20米；红军在杨梅分局北侧配置2辆CM-12主战坦克，呈并列阵型，间距15米。']
# from data.inputs_unique import orders as orders0
# from data.inputs_unique_goodloc import orders as orders1
from data.inputs_unique import orders as orders0
from data.inputs_unique_goodloc import orders as orders1
orders = list(orders1.values())
# orders = list(orders1.values())
for bad in bad_loc:
    if isinstance(bad, list):
        if bad[0] in orders:
            orders.remove(bad[0])
    else:
        if bad in orders:
            orders.remove(bad)
def merge_list(l1, l2, path):
    both = list(set(l1+l2))
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(both):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
save_list(orders, 'data/inputs_unique_goodloc.py')
# len(orders)

## 随机打乱指令

In [1]:
from data.inputs_unique import orders as raw_orders
import random
orders0 = list(raw_orders.values())
def unique_list(l):
    return list(set(l))

# 以10个为单位，每10个指令包含8个单色指令，2个双色指令
def decompose(orders):
    n = len(orders)
    x=8
    y=2
    monos = []
    polys = []
    for i in range(0, n, 10):
        monos+=orders0[i:i+x]
        polys+=orders0[i+x:i+x+y]
    return monos, polys

monos, polys = decompose(orders0)
random.shuffle(monos)
random.shuffle(polys)

def compose(monos, polys):
    n = len(monos+polys)
    x=8
    y=2
    m=0
    p=0
    orders = []
    for i in range(0, n, 10):
        try:
            orders[i:i+10] = monos[m:m+x]+polys[p:p+y]
            m+=x
            p+=y
        except:
            orders[i:i+10] = monos[m:]+polys[p:]
    return orders

orders = compose(monos, polys)

def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
save_list(orders, 'data/inputs_unique_exp.py')

: 

## 制作复杂部署指令集

In [1]:
import random
import json
def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")
from data.zh_inputs_unique import orders as orders_raw
two_n = 50
two = []
three_n = 50
three = []
four_n = 50
four = []
five_n = 50
five = []
ten_n = 50
ten = []
twenty_n = 50
twenty = []
thirty_n = 50
thirty = []
fifty_n = 50
fifty = []
hundred_n = 50
hundred = []

file_list = list(orders_raw.values())
for i in range(two_n):
    two.append('; '.join(random.sample(file_list, 2)))
        
for i in range(three_n):
    three.append('; '.join(random.sample(file_list, 3)))

for i in range(four_n):
    four.append('; '.join(random.sample(file_list, 4)))

for i in range(five_n):
    five.append('; '.join(random.sample(file_list, 5)))

for i in range(ten_n):
    ten.append('; '.join(random.sample(file_list, 10)))

for i in range(twenty_n):
    twenty.append('; '.join(random.sample(file_list, 20)))

for i in range(thirty_n):
    thirty.append('; '.join(random.sample(file_list, 30)))

for i in range(fifty_n):
    fifty.append('; '.join(random.sample(file_list, 50)))

for i in range(hundred_n):
    hundred.append('; '.join(random.sample(file_list, 100)))

save_list(list(set(file_list)), 'data/zh_inputs_unique_1.py')
save_list(list(set(two)), 'data/zh_inputs_unique_2.py')
save_list(list(set(three)), 'data/zh_inputs_unique_3.py')
save_list(list(set(four)), 'data/zh_inputs_unique_4.py')
save_list(list(set(five)), 'data/zh_inputs_unique_5.py')
save_list(list(set(ten)), 'data/zh_inputs_unique_10.py')
save_list(list(set(twenty)), 'data/zh_inputs_unique_20.py')
save_list(list(set(thirty)), 'data/zh_inputs_unique_30.py')
save_list(list(set(fifty)), 'data/zh_inputs_unique_50.py')
save_list(list(set(hundred)), 'data/zh_inputs_unique_100.py')
# save_list(list(list(set(random.sample(file_list, 10)))+list(set(random.sample(two, 10)))+list(set(random.sample(three, 10)))+list(set(random.sample(four, 10)))+list(set(random.sample(five, 10)))), 'data/geospatial_complex.py')


: 

# 把伪单一指令改为真单一指令

In [9]:
from data.zh_inputs_unique import orders as orders_raw
# from data.inputs_unique import orders as orders_raw
import re

def save_list(l, path):
    with open(path, 'w') as f:
        f.write("orders = {\n")
        for key, value in enumerate(l):
            f.write(f"    \"order_{repr(key)}\": {repr(value)},\n")  # 使用 repr 确保正确格式化
        f.write("}\n")

orders = list(orders_raw.values())
real_orders = []
for order in orders:
    real_order = [seg.strip() for seg in re.split(r'[;；]', order) if seg.strip()]
    # real_order = [seg.strip() for seg in re.split(r'\bwhile\b', order, flags=re.IGNORECASE) if seg.strip()]
    real_orders.extend(set(real_order))
save_list(real_orders, 'data/zh_inputs_unique.py')
